In [54]:
import pandas as pd
import numpy as np
import jieba 

from keras.layers import Dense, Input, Flatten,Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding, concatenate
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model


In [2]:
fneg = open('neg.txt','r',encoding='gb18030')
fpos = open('pos.txt','r',encoding='gb18030')

In [3]:
neg = []
pos = []
for line in fneg:
    neg.append(line)
for line in fpos:
    pos.append(line)

In [8]:
neg_df = pd.DataFrame(neg)
pos_df = pd.DataFrame(pos)

In [13]:
pos_df.tail()

,0
4995,早餐比较差，别的都还好啦。离香洲汽车站满近的，步行不到10分钟也就到了。这点满方便的。\n
4996,晚上到的深圳机场，打的到酒店半个小时左右，120块的样子.酒店CHECK IN 还算快.但是...
4997,房间整体感觉十分干净，环境也比较清静，虽然临街一边的客房要相对吵一些，但是隔音做的不错\n
4998,"八月初入住两天,感觉极好.幽雅的庭院式格局与众多的现代高楼酒店相比,给人一种苏州园林式的恬静..."
4999,大连很不错的酒店了，不过差点和南山花园那个搞错0.0


In [15]:
#合并语料

pn = pd.concat([neg_df, pos_df], ignore_index=True)
neg_len = len(neg_df)
pos_len = len(pos_df)

In [16]:
#定义分词函数

cw = lambda x: list(jieba.cut(x))
pn['words'] = pn[0].apply(cw)

Building prefix dict from the default dictionary ...
I0728 08:21:20.159217  7936 __init__.py:111] Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\Andy\AppData\Local\Temp\jieba.cache
I0728 08:21:23.505533  7936 __init__.py:145] Dumping model to file cache C:\Users\Andy\AppData\Local\Temp\jieba.cache
Loading model cost 3.620 seconds.
I0728 08:21:23.783891  7936 __init__.py:163] Loading model cost 3.620 seconds.
Prefix dict has been built succesfully.
I0728 08:21:23.787896  7936 __init__.py:164] Prefix dict has been built succesfully.


In [17]:
pn.head()

,0,words
0,专程从成都来绵阳看第四届科博会，在万达广场吃了晚饭都快九点了，携程上匆忙订了王子大酒店冲着它...,"[专程, 从, 成都, 来, 绵阳, 看, 第四届, 科博会, ，, 在, 万达, 广场, ..."
1,我本人办的有锦江之星的会员卡，相对来说锦江住的比较多，石家庄的锦江我只住过平安大街店，感觉非...,"[我, 本人, 办, 的, 有, 锦江, 之星, 的, 会员卡, ，, 相对来说, 锦江, ..."
2,"可能是以上海地区而言不算贵吧,但是一切都乏善可陈,就是说不出甚么特别不好的,但是也说不出一点...","[可能, 是, 以, 上海地区, 而言, 不算, 贵, 吧, ,, 但是, 一切, 都, 乏..."
3,"建议携程取消推荐该酒店,环境很差,楼下有迪厅,不仅有噪音,整个房间都在颤动,根本无法入睡!另...","[建议, 携程, 取消, 推荐, 该, 酒店, ,, 环境, 很差, ,, 楼下, 有, 迪..."
4,"周围环境差,房间设施陈旧.房间小.服务员懒散.去餐厅很不方便,路线复杂.早餐贵,而且没有什么...","[周围环境, 差, ,, 房间, 设施, 陈旧, ., 房间, 小, ., 服务员, 懒散,..."


In [18]:
#统计一行最多有多少个单词

max_document_length = max([len(x) for x in pn['words']])
max_document_length

1287

In [19]:
#将一行最大单词数设为1000
max_document_length = 1000

In [20]:
texts = [' '.join(x) for x in pn['words']]

In [25]:
texts[0]

'专程 从 成都 来 绵阳 看 第四届 科博会 ， 在 万达 广场 吃 了 晚饭 都 快 九点 了 ， 携程 上 匆忙 订 了 王子 大酒店 冲着 它 是 四星级 ， 门脸 很 阔气 大堂 够 大气 ， 进 了 客房 略微 失望 ， 洗个 澡 吓死 宝宝 ， 看 图 说话 。 是 我 图 便宜 就 选到 了 250 一晚 的 标间 ， 怪 自己 了 ， 这 酒店 评级 标准 咋 能 这样 呢 ， 老 四星 也 太 追求 环保 了 吧 ， 该换 的 不换 该 清理 的 不 清理 这是 评级 标准 吗 ？   想 住 绵阳 四星 的 朋友 看 过来 ， 要 住 就 住 他家 五百 八百 的 房 吧 ， 二百五十 多 的 就 别来 了 ， 去汉庭 7 天 算了   还有 停车 过夜 ， 前台 美女 说 ： 我们 星级 酒店 没有 免费 停车 的 。 。 。   此处 省略 一万 字 \n'

In [31]:
#实例化分词器
tokenizer = Tokenizer(num_words=30000)
#建立词典
tokenizer.fit_on_texts(texts)
#将词频和序号对应
sequences = tokenizer.texts_to_sequences(texts)
#将序列设置为1000的长度，超过1000的部分舍弃，不足的补0
sequences = pad_sequences(sequences, maxlen=1000, padding='post')
#sequences = np.array(sequences)
type(sequences)

numpy.ndarray

In [29]:
sequences

array([[ 8369,   137,  1243, ...,     0,     0,     0],
       [    9,   494,   839, ...,     0,     0,     0],
       [  183,     6,   567, ...,     0,     0,     0],
       ...,
       [    7,   321,    33, ...,     0,     0,     0],
       [10142, 11861,    24, ...,     0,     0,     0],
       [ 1401,     8,    17, ...,     0,     0,     0]])

In [32]:
#词对应编号的词典

dict_text = tokenizer.word_index
dict_text['天']

346

In [33]:
sequences[0]

array([ 8369,   137,  1243,    93,  4981,   150, 14895, 14896,     1,
          12,  4982,   783,   111,     4,  1683,    13,   535,  1991,
           4,     1,    47,    59,  2156,   164,     4,  7050,   603,
        2526,   445,     6,   374,     1,  4983,     8,  8370,   123,
         585,  2223,     1,   490,     4,   220,  3973,   362,     1,
        2527,  1321,  4984,  2086,     1,   150,  2284,   838,     3,
           6,     9,  2284,   258,    22, 14897,     4,  4242,   454,
           2,   377,     1,  2224,   138,     4,     1,    80,     5,
        3708,   205,  2034,   100,   122,   324,     1,   298,   268,
          10,    82,  5481,  2985,     4,   115,     1,  5482,     2,
        3974,   166,  1432,     2,    15,  1432,   472,  3708,   205,
         340,    85,   190,    21,  4981,   268,     2,   167,   150,
         550,     1,    38,    21,    22,    21,  2986, 10471,  6170,
           2,    64,   115,     1, 14898,    89,     2,    22, 10472,
           4,     1,

In [36]:
#这里的x是句子中每个分词对应的word_index的列表

#定义标签
positive_labels = [[0, 1] for _ in range(pos_len)]
negative_labels = [[1, 0] for _ in range(neg_len)]
y = np.concatenate([positive_labels, negative_labels], 0)
#打乱数据
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = sequences[shuffle_indices]
y_shuffled = y[shuffle_indices]

#数据集切分
test_sample_index = -1 * int(0.15 * float(len(y)))
x_train, x_test = x_shuffled[:test_sample_index], x_shuffled[test_sample_index:]
y_train, y_test = y_shuffled[:test_sample_index], y_shuffled[test_sample_index:]


In [43]:
#定义模型
#模型输入
sequence_input = Input(shape=(1000,))
#Embedding层，30000表示30000个词，每个词对应向量128维，序列长度1000
embedding_layer = Embedding(
    30000,  #生成一个30000*128的矩阵
    128, 
    input_length=1000
)
embedding_sequences = embedding_layer(sequence_input)

#卷积核大小为3

cnn1 = Conv1D(filters=32, kernel_size=3, activation='relu')(embedding_sequences)
cnn1 = MaxPooling1D(pool_size=5)(cnn1)
cnn1 = Conv1D(filters=32, kernel_size=3, activation='relu')(cnn1)
cnn1 = MaxPooling1D(pool_size=5)(cnn1)
cnn1 = Conv1D(filters=32, kernel_size=3, activation='relu')(cnn1)
cnn1 = MaxPooling1D(pool_size=37)(cnn1)
cnn1 = Flatten()(cnn1)
                                                            
#卷积核大小为4

cnn2 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding_sequences)
cnn2 = MaxPooling1D(pool_size=5)(cnn2)
cnn2 = Conv1D(filters=32, kernel_size=4, activation='relu')(cnn2)
cnn2 = MaxPooling1D(pool_size=5)(cnn2)
cnn2 = Conv1D(filters=32, kernel_size=4, activation='relu')(cnn2)
cnn2 = MaxPooling1D(pool_size=36)(cnn2)
cnn2 = Flatten()(cnn2) 
                                                            
#卷积核大小为5

cnn3 = Conv1D(filters=32, kernel_size=5, activation='relu')(embedding_sequences)
cnn3 = MaxPooling1D(pool_size=5)(cnn3)
cnn3 = Conv1D(filters=32, kernel_size=5, activation='relu')(cnn3)
cnn3 = MaxPooling1D(pool_size=5)(cnn3)
cnn3 = Conv1D(filters=32, kernel_size=5, activation='relu')(cnn3)
cnn3 = MaxPooling1D(pool_size=35)(cnn3)
cnn3 = Flatten()(cnn3)

#合并
merge = concatenate([cnn1, cnn2, cnn3], axis=1)
#全连接层

x = Dense(128, activation='relu')(merge)
x = Dropout(0.5)(x)
preds = Dense(2, activation='softmax')(x)

model = Model(sequence_input, preds)

In [44]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 1000, 128)    3840000     input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_19 (Conv1D)              (None, 998, 32)      12320       embedding_3[0][0]                
__________________________________________________________________________________________________
conv1d_22 (Conv1D)              (None, 997, 32)      16416       embedding_3[0][0]                
__________________________________________________________________________________________________
conv1d_25 

In [45]:
#编译和训练
model.compile(
    optimizer='adam', 
    loss='categorical_crossentropy',
    #loss = 'mse',
    metrics=['accuracy'])

W0728 16:13:45.152997  7936 deprecation_wrapper.py:119] From C:\Conda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0728 16:13:45.219060  7936 deprecation_wrapper.py:119] From C:\Conda3\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [46]:
model.fit(x_train, y_train, batch_size=32, epochs=10)

W0728 16:14:14.525673  7936 deprecation.py:323] From C:\Conda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
8500/8500 [==============================] - 123s 15ms/step - loss: 0.4193 - acc: 0.7884
Epoch 2/10
8500/8500 [==============================] - 119s 14ms/step - loss: 0.1506 - acc: 0.9455
Epoch 3/10
8500/8500 [==============================] - 117s 14ms/step - loss: 0.0491 - acc: 0.9859
Epoch 4/10
8500/8500 [==============================] - 116s 14ms/step - loss: 0.0186 - acc: 0.9951
Epoch 5/10
8500/8500 [==============================] - 116s 14ms/step - loss: 0.0139 - acc: 0.9960
Epoch 6/10
8500/8500 [==============================] - 116s 14ms/step - loss: 0.0052 - acc: 0.9986
Epoch 7/10
8500/8500 [==============================] - 116s 14ms/step - loss: 0.0019 - acc: 0.9996
Epoch 8/10
8500/8500 [==============================] - 117s 14ms/step - loss: 7.2478e-04 - acc: 0.9999
Epoch 9/10
8500/8500 [==============================] - 116s 14ms/step - loss: 0.0218 - acc: 0.9936
Epoch 10/10
8500/8500 [==============================] - 116s 14ms/step - loss: 0.0055 - acc: 0.

In [51]:
#预测函数

def predict(text):
    cw = list(jieba.cut(text))
    word_id = []
    for word in cw:
        try:
            temp = dict_text(word)
            word_id.append(temp)
        except:
            word_id.append(0)
    word_id = np.array(word_id)
    word_id = word_id[np.newaxis,:]
    sequences = pad_sequences(word_id,maxlen=1000,padding='post')
    print(model.predict(sequences))
    result = np.argmax(model.predict(sequences))
    if result == 1:
        print("Negative")
    else:
        print("Positive")
            

In [52]:
predict("这里的天气真是糟透了，店员脑子瓦特了")

[[0.01159209 0.9884079 ]]
Negative


In [53]:
loss, accuracy = model.evaluate(x_train, y_train)
print("Loss:", loss)
print("Accuracy:", accuracy)

8500/8500 [==============================] - 15s 2ms/step
Loss: 0.00023811632527555475
Accuracy: 1.0
